<div style="height: auto; position: relative; background: #f0f0f0; margin-bottom: 20px; padding: 30px 0;">
    <div style="display: inline-block; background: #003247; color: #fff; font-size: 26px; font-weight: 700; padding: 30px;">
        PS5E11: Predicting Loan Payback | EDA 📊 | FE ⚙️|  CatBoost 🚀 | Optuna 🔬 
    </div>
</div>

<div style="color: white;
    display: flex;
    justify-content: center;
    align-items: center;
    border-radius: 5px;
    background-color: #003247;
    font-size: 220%;
    font-family: Nexa;
    letter-spacing: 0.5px;
    min-height: 100px;
    padding: 20px; 
    width: 100%;
    box-sizing: border-box;">
    <h1 id = "description-goal"
        style=" color: white;
        margin: 0;
        text-align: center;
        width: 100%;">
        <b>Description & Goal 📣</b>
    </h1>
</div>

# **Overview** 

    
This task involves predicting the probability that a borrower will pay back their loan based on a comprehensive features encompassing socioeconomic demographics, financial indicators, and institutional risk classifications. The problem represents a canonical binary classification challenge within credit risk analytics, where accurate prediction directly impacts financial institution's capital allocation efficiency and systemic stability. The primary challenge in credit risk assessment emerges from the absence of established credit histories, which fundamentally impedes the determination of conditional risk categorization and the assignment of meaningful credit scores. In such a case, one should rely on a high-quality and well-justified feature engineering section and meticulously select the derived features for subsequent model selection and training for predictions.



Submissions are scored on [area under the ROC curve](https://en.wikipedia.org/wiki/Receiver_operating_characteristic) between the predicted probability and the observed target. The **AUC ROC** quantifies a model’s ability to distinguish between positive and negative classes by measuring the probability that a randomly selected positive instance is ranked higher than a randomly selected negative instance. It is defined as:

\\[
\text{AUC} = \frac{|{(i,j) \in S : y_i = 1, y_j = 0 \text{ and } \hat{y}_i > \hat{y}_j}|}{|S|}
\\]


<div style="display: flex; align-items: flex-start;">
  <div style="flex: 1; max-width: 35%; padding-right: 20px;">
    <img src="https://s3.amazonaws.com/media-p.slid.es/uploads/1094055/images/10690666/pasted-from-clipboard.png" 
         style="max-height: 400px; max-width: 100%; object-fit: contain;">
  </div>
  <div style="flex: 2;">
    <p>where:</p>
    <ul>
      <li>$y_i, y_j$ — true binary labels (1 = positive class, 0 = negative class),</li>
      <li>$\hat{y}_i, \hat{y}_j$ — predicted probabilities for the positive class,</li>
      <li>$S$ — set of all pairs with $y_i \neq y_j$,</li>
      <li>$|S|$ — total number of such pairs.</li>
    </ul>
    <p>The <b>AUC</b> reflects the proportion of correctly ordered pairs: for every pair of instances where the true label of $i$ is positive and $j$ is negative, the model's predicted probability for $i$ should exceed that for $j$.</p>
    <p>The metric score ranges from <b>`0`</b> to <b>`1`</b>:</p>
    <ul>
      <li><b>`0.5`</b> — the model performs no better than random guessing (equivalent to a diagonal ROC curve);</li>
      <li><b>`1.0`</b> — the model perfectly separates the classes (all positive instances are ranked above negative ones);</li>
      <li><b>`&lt; 0.5`</b> — the model's predictions are inversely correlated with the true labels (worse than random; flipping predictions would improve performance).</li>
    </ul>
    <p><b>AUC</b> is <i>threshold-agnostic</i>, meaning it evaluates model performance across all possible classification thresholds, making it ideal for imbalanced datasets where accuracy or precision may be misleading.</p>
  </div>
</div>

<div style="color: white;
    display: flex;
    justify-content: center;
    align-items: center;
    border-radius: 5px;
    background-color: #003247;
    font-size: 220%;
    font-family: Nexa;
    letter-spacing: 0.5px;
    min-height: 100px;
    padding: 20px; 
    width: 100%;
    box-sizing: border-box;">
    <h1 id = "table-of-contents"
        style=" color: white;
        margin: 0;
        text-align: center;
        width: 100%;">
        <b>Table of Contents 🔠</b>
    </h1>
</div>

* [1. Description & Goal 📣](#description-goal)
* [2. Table of Contents 🔠](#table-of-contents)
* [3. Import 📚](#import)
* [4. $\lambda$uxiliary Functions ⚗️](#auxiliary-func)
* [5. Load Data 💾](#data-loading)
* [5.1 Data Understanding 🗃️🧐](#data-understanding)
* [5.2 HeatMap 🔥🗺️ + Clustering 🧶 = Cluster Map 🏜️](#cluster-map)
* [6. EDA 🕵 | 📊](#eda)
* [7. Feature Engineering 🛠️ | ⚙️](#fe)
* [8. Modeling 🤖](#modeling)
* [8.1 Optuna 🔬](#optuna)
* [8.2 CatBoost 🐈 | 🚀](#catboost)
* [8.3 XgBoost 🤖](#xgb)
* [8.4 LightGBM 👾](#lgb)
* [8.5 Stacking 🧱|🧠](#stack)
* [9. Submission ✔️](#submission)
* [10. Summary 💭](#summary)
* [11. References 📜](#refs)

<div style="color: white;
    display: flex;
    justify-content: center;
    align-items: center;
    border-radius: 5px;
    background-color: #003247;
    font-size: 220%;
    font-family: Nexa;
    letter-spacing: 0.5px;
    min-height: 100px;
    padding: 20px; 
    width: 100%;
    box-sizing: border-box;">
    <h1 id = "import"
        style=" color: white;
        margin: 0;
        text-align: center;
        width: 100%;">
        <b>Import 📚</b>
    </h1>
</div>

In [ ]:
import os
import time
import joblib
import random
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.colors import Normalize
from matplotlib.patches import Rectangle
from scipy.stats import gaussian_kde, probplot
from statsmodels.graphics.mosaicplot import mosaic
from sklearn.preprocessing import PolynomialFeatures
from scipy.stats import chi2_contingency, ttest_ind, mannwhitneyu


import shap
import optuna
import joblib
import xgboost as xgb
import lightgbm as lgb
from sklearn.pipeline import Pipeline
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, PowerTransformer
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score



pd.reset_option('display.max_columns')
warnings.filterwarnings('ignore')
%matplotlib inline

<div style="color: white;
    display: flex;
    justify-content: center;
    align-items: center;
    border-radius: 5px;
    background-color: #003247;
    font-size: 220%;
    font-family: Nexa;
    letter-spacing: 0.5px;
    min-height: 100px;
    padding: 20px; 
    width: 100%;
    box-sizing: border-box;">
    <h1 id = "auxiliary-func"
        style=" color: white;
        margin: 0;
        text-align: center;
        width: 100%;">
        <b>$\lambda$uxiliary Functions ⚗️</b>
    </h1>
</div>

Functions for preprocesssing input, losses, visualization, setting seed for consistent results and etc.

In [ ]:
# setting seed
SEED = 64911002

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
seed_everything(SEED)


def nan_check(df):
    total_entries = df.shape[0] * df.shape[1]
    missing_entries_max = df.isnull().sum().sum()
    missing_entries_max_percentage = (missing_entries_max / total_entries) * 100
    print(f"Total entries in the dataset: {total_entries}")
    print(f"Maximum missing values in the dataset: {missing_entries_max}")
    print(f"Percentage of maximum missing values in the dataset: {missing_entries_max_percentage:.2f}%")
    return df.isna().sum()


def feature_value_counts(df, columns_to_include=None):
    """
    Outputs the Distribution of values for each attribute in the DataFrame.
    
    Parameters:
    df (pd.DataFrame): Input DataFrame
    columns_to_include (list, optional): List of column names to process. 
                                        If None, all columns will be processed.
    """
    # Определяем список колонок для обработки
    columns = columns_to_include if columns_to_include is not None else df.columns
    
    for column in columns:
        if column not in df.columns:
            print(f"Warning: Column '{column}' not found in DataFrame. Skipping.")
            continue
            
        print(f"Distribution of '{column}' feature:")
        print(df[column].value_counts())
        print("\n")


def feature_distribution(df, columns_to_visualize=None, exclude_from_visualization=None):
    """
    Visualizes feature distributions with division into numerical and categorical ones
    """
    if columns_to_visualize is None:
        columns_to_visualize = df.columns.tolist()
    if exclude_from_visualization:
        columns_to_visualize = [col for col in columns_to_visualize if col not in exclude_from_visualization]
    num_cols = df[columns_to_visualize].select_dtypes(include=np.number).columns.tolist()
    cat_cols = df[columns_to_visualize].select_dtypes(exclude=np.number).columns.tolist()
    
    # numerical
    if num_cols:
        fig_num, axes_num = plt.subplots(
            len(num_cols), 3, 
            figsize=(24, 6 * len(num_cols)),
            squeeze=False
        )
        
        for i, col in enumerate(num_cols):
            ax_hist = axes_num[i, 0]
            ax_box = axes_num[i, 1]
            ax_stat = axes_num[i, 2]
            sns.histplot(df[col], kde=True, ax=ax_hist, bins=30, color='steelblue')
            mean_val = df[col].mean()
            std_val = df[col].std()
            ax_hist.axvline(mean_val, color='darkorange', linestyle='--', label=f'Mean: {mean_val:.2f}')
            ax_hist.axvline(mean_val - std_val, color='red', linestyle=':', label='±1 Std')
            ax_hist.axvline(mean_val + std_val, color='red', linestyle=':')
            ax_hist.legend()
            
            # box-plot with outliers detection
            sns.boxplot(y=df[col], ax=ax_box, color='steelblue')
            q1, q3 = df[col].quantile([0.25, 0.75])
            iqr = q3 - q1
            lower, upper = q1 - 1.5*iqr, q3 + 1.5*iqr
            
            if df[col].min() < lower:
                ax_box.add_patch(Rectangle(
                    (-0.4, df[col].min()), 0.8, lower - df[col].min(),
                    edgecolor='red', linestyle='--', fill=False))
            if df[col].max() > upper:
                ax_box.add_patch(Rectangle(
                    (-0.4, upper), 0.8, df[col].max() - upper,
                    edgecolor='red', linestyle='--', fill=False))
            
            # stats
            stats = [
                f"Mean: {mean_val:.2f}",
                f"Std: {std_val:.2f}",
                f"Min: {df[col].min():.2f}",
                f"Max: {df[col].max():.2f}",
                f"Skew: {df[col].skew():.2f}",
                f"Kurtosis: {df[col].kurtosis():.2f}"
            ]
            ax_stat.axis('off')
            ax_stat.text(0.1, 0.5, '\n'.join(stats), fontsize=12, va='center')
        
        plt.tight_layout()
        plt.show()
    
    # categorical
    if cat_cols:
        fig_cat, axes_cat = plt.subplots(
            len(cat_cols), 2, 
            figsize=(20, 5 * len(cat_cols)),
            squeeze=False
        )
        
        for i, col in enumerate(cat_cols):
            ax_count = axes_cat[i, 0]
            ax_top = axes_cat[i, 1]
            
            # plot n-categories
            top_categories = df[col].value_counts().nlargest(20).index
            df_plot = df[col].apply(lambda x: x if x in top_categories else 'Other')
            sns.countplot(x=df_plot, ax=ax_count, palette='coolwarm')
            ax_count.tick_params(axis='x', rotation=90)
            
            # plot 5-categories
            top5 = df[col].value_counts().nlargest(5)            
            sns.barplot(x=top5.values, y=top5.index, ax=ax_top, palette='coolwarm', orient='h')
        plt.tight_layout()
        plt.show();

<div style="color: white;
    display: flex;
    justify-content: center;
    align-items: center;
    border-radius: 5px;
    background-color: #003247;
    font-size: 220%;
    font-family: Nexa;
    letter-spacing: 0.5px;
    min-height: 100px;
    padding: 20px; 
    width: 100%;
    box-sizing: border-box;">
    <h1 id = "data-loading"
        style=" color: white;
        margin: 0;
        text-align: center;
        width: 100%;">
        <b>Load Data 💾</b>
    </h1>
</div>

In [ ]:
df = pd.read_csv("/kaggle/input/playground-series-s5e11/train.csv")
test = pd.read_csv("/kaggle/input/playground-series-s5e11/test.csv")

In [ ]:
df.head()

In [ ]:
test.head()

Based on  [source](https://www.kaggle.com/datasets/nabihazahid/loan-prediction-dataset-2025/data):


1. `id` - an unique identifier for each record;
2. `annual_income` - borrower's yearly income;
3. `debt_to_income_ratio` - ratio of borrower’s debt to their income. Lower = better;
4. `credit_score` - credit bureau score (e.g., FICO). Higher = less risky;
5. `loan_amount` - amount of loan taken;
6. `interest_rate` - loan par annual interest rate (%);
7. `gender` - borrower's gender (Male/Female);
8. `marital_status` - marital status (Single, Married, Divorced);
9. `education_level` -  education level (High School, Bachelor, Master, PhD);
10. `employment_status` - current employment type (Employed, Self-Employed, Unemployed);
11. `loan_purpose` - loan purpose (Car, Education, Home, Medical, etc.);
12. `grade_subgrade` - risk category assigned to loan (A1, B2, etc.);
13. `loan_paid_back` - target variable (`0` = borrower defaulted (did not repay fully), `1` = borrower paid loan in full)


Each row represents an individual loan applicant's financial and demographic information, with features describing creditworthiness indicators and a binary target variable **`loan_paid_back`** indicating whether the borrower fully repaid their loan **`1`** or defaulted **`0`**.

<div style="color: white;
    display: flex;
    justify-content: center;
    align-items: center;
    border-radius: 5px;
    background-color: #003247;
    font-size: 220%;
    font-family: Nexa;
    letter-spacing: 0.5px;
    min-height: 100px;
    padding: 20px; 
    width: 100%;
    box-sizing: border-box;">
    <h2 id = "data-understanding"
        style=" color: white;
        margin: 0;
        text-align: center;
        width: 100%;">
        <b>Data Understanding 🗃️🧐</b>
    </h2>
</div>

## Check the data types

In [ ]:
df.info()

In [ ]:
test.info()

The dataset contains both categorical and numerical features

## Check the shape of data

In [ ]:
print(f"Train dataset has {df.shape[0]} rows and {df.shape[1]} columns")
print(f"Test dataset has {test.shape[0]} rows and {test.shape[1]} columns")

## Check the missing values

In [ ]:
 nan_check(df)

In [ ]:
 nan_check(test)

None NaN values are seen in both datasets

## Check the unique values

In [ ]:
unique = df.select_dtypes(exclude=['object']).nunique()
unique = unique[unique == 1]
print(f"Columns with only one unique value: {unique}")

None features with only one unique value

In [ ]:
df.describe(include = 'all')

In [ ]:
test.describe(include = 'all')

The comparison between training and test sets reveals critical distributional shifts that warrant attention for model robustness:

1. Interest Rate Drift: Evolving Risk Appetite
A notable increase in the maximum interest rate is observed in the test set (`21.29%` VS `20.99%` in training);

2. Annual Income Extrapolation Risk
The test set shows a `3.34%` lower maximum annual income (`380654`) compared to the training set (`393381`). This indicates the absence of ultra-high-income clients in the test data. Consequently, the model has not been validated on this segment and may demonstrate instability when making predictions for new, affluent clients, as it must extrapolate beyond the income range it was assessed on;

3. Debt-to-Income Ratio Stability
In contrast to the other features, the Debt-to-Income Ratio shows remarkable stability. All key statistics are identical up to the fourth decimal place between the train and test sets. This indicates a high-quality, stratified split for this critical risk factor, ensuring that the model's performance on DTI is likely to be reliable and consistent.

<div style="color: white;
    display: flex;
    justify-content: center;
    align-items: center;
    border-radius: 5px;
    background-color: #003247;
    font-size: 220%;
    font-family: Nexa;
    letter-spacing: 0.5px;
    min-height: 100px;
    padding: 20px; 
    width: 100%;
    box-sizing: border-box;">
    <h2 id = "cluster-map"
        style=" color: white;
        margin: 0;
        text-align: center;
        width: 100%;">
        <b>HeatMap 🔥🗺️ + Clustering 🧶 = Cluster Map 🏜️</b>
    </h2>
</div>

In [ ]:
numeric_df = df.select_dtypes(include=['number'])
corr_matrix = numeric_df.corr()
sns.clustermap(corr_matrix, annot = True, fmt = '.2f', linewidths = .5, cmap='coolwarm', figsize = (16, 8))
plt.show();

The analysis of numeric features revealed **five** distinct clusters:

1. Credit-Interest Cluster (`credit_score`, `interest_rate`) exhibits a strong inverse relationship (`−0.54`), indicating that lenders systematically adjust interest rates as a function of creditworthiness. Higher `credit_score` values correlate with lower `interest_rate`, reflecting risk-based pricing mechanisms.
2. Debt-Repayment Cluster (`debt_to_income_ratio`, `loan_paid_back`) has a moderate negative correlation (`−0.34`). This suggests that borrowers with higher `debt-to-income` ratios are less likely to fully repay loans, highlighting a potential causal link between financial burden and repayment behavior. This makes perfect business sense: as a client's debt burden increases relative to their income, their ability to repay a new loan decreases significantly.
3. Isolated Features (`annual_income`, `loan_amount`) operate independently, showing negligible correlations (`< 0.06`) with all other variables. These features lack strong structural ties to the identified clusters, implying they capture distinct or unobserved dimensions of borrower risk.
4. Credit-Quality Cluster (`credit_score`, `loan_paid_back`) is the strongest positive correlation (`0.23`). Clients with higher `credit_scores` are statistically more likely to repay their loans, which aligns with global lending practices.
5. Risk-Based Pricing Cluster (`interest_rate`, `loan_paid_back`) (`-0.13`) suggests that interest rate alone is a less powerful predictor. Its effect is likely indirect and mediated through the credit score.




The dataset provides two very strong, logical signals for predicting loan repayment (`debt_to_income_ratio` and `credit_score`). Although `annual_income` and `loan_amount` show no direct linear relationship, they should not be discarded. It's plausible that a large loan is risky for a low-income borrower, even if the two features individually are not predictive. The low correlations between most features simplify the modeling process by avoiding multicollinearity issues. The lack of direct correlation from other financial features is not a drawback but rather a clear directive to explore more sophisticated feature engineering, particularly through **interaction terms** and **non-linear transformations**, to unlock their predictive potential.

<div style="color: white;
    display: flex;
    justify-content: center;
    align-items: center;
    border-radius: 5px;
    background-color: #003247;
    font-size: 220%;
    font-family: Nexa;
    letter-spacing: 0.5px;
    min-height: 100px;
    padding: 20px; 
    width: 100%;
    box-sizing: border-box;">
    <h1 id = "eda"
        style=" color: white;
        margin: 0;
        text-align: center;
        width: 100%;">
        <b>EDA 🕵 | 📊</b>
    </h1>
</div>



## Distribution of the target 

In [ ]:
loan_counts = df['loan_paid_back'].value_counts()

labels = ['Paid (1)', 'Defaulted (0)']
sizes = loan_counts.values
percentages = [f'{(count/sum(sizes))*100:.1f}%' for count in sizes]
plt.figure(figsize=(16, 8))
colors = sns.color_palette('coolwarm', n_colors=2)
colors = colors[::-1]

# pie chart
wedges, texts, autotexts = plt.pie(sizes, 
                                  labels=labels, 
                                  autopct='%1.1f%%',
                                  colors=colors,
                                  startangle=90,
                                  explode=(0.05, 0.05),  
                                  textprops={'fontsize': 14, 'fontweight': 'bold'},
                                  wedgeprops={'edgecolor': 'black', 'linewidth': 1.5})

for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontweight('bold')
    autotext.set_fontsize(16)
    
plt.title('Loan Repayment Status Distribution', 
          fontsize=24, 
          fontweight='bold', 
          pad=20,
          color='#2c3e50')

legend_labels = [
    f'Paid Loans: {sizes[0]:,} ({percentages[0]})',
    f'Defaulted Loans: {sizes[1]:,} ({percentages[1]})'
]

plt.legend(wedges, legend_labels,
          title="Loan Status Details",
          loc="center left",
          bbox_to_anchor=(1, 0, 0.5, 1),
          fontsize=14,
          title_fontsize=16,
          frameon=True,
          framealpha=0.9,
          edgecolor='#2c3e50')

total_loans = sum(sizes)
plt.text(0, 0, f'Total Loans\n{total_loans:,}', 
         ha='center', 
         va='center', 
         fontsize=18, 
         fontweight='bold',
         color='#2c3e50',
         bbox=dict(facecolor='white', alpha=0.8, edgecolor='#2c3e50', boxstyle='round,pad=1'))

plt.figtext(0.5, 0.02, 
           f'Insight: With a {percentages[0]} repayment rate, the data shows healthy performance\n'
           f'but the {percentages[1]} default rate represents significant recoverable risk exposure',
           ha='center', 
           fontsize=14, 
           fontstyle='italic',
           bbox=dict(facecolor='lightyellow', alpha=0.9, edgecolor='orange', boxstyle='round,pad=0.5'))
plt.tight_layout(rect=[0, 0.05, 0.85, 1])

The pie chart presents a significant dataset of `593994` loan records, revealing a class distribution where `79.9%` (`474494` loans) were fully repaid while `20.1%` (`119500` loans) defaulted. This `4:1` **class imbalance** has profound implications for model development and evaluation.

The `20.1%` default rate within this substantial dataset presents both challenges and opportunities for predictive modeling. While the class imbalance complicates traditional modeling approaches, it underscores the critical importance of proper evaluation metrics and business-aligned model development. Rather than optimizing for overall accuracy, practitioners should focus on identifying the most at-risk borrowers, where even modest improvements in identifying default cases can yield substantial financial benefits. Future work must prioritize techniques that address class imbalance through cost-sensitive learning, threshold optimization, and appropriate evaluation metrics. The high default rate suggests this dataset could provide valuable insights into risk factors that standard lending models may overlook, potentially leading to more sophisticated and profitable risk assessment frameworks that balance portfolio growth with risk management.

## Distribution of numeric features

In [ ]:
numeric_features = ['annual_income', 'debt_to_income_ratio', 'credit_score', 'loan_amount', 'interest_rate']
feature_value_counts(df, columns_to_include = numeric_features)

In [ ]:
feature_distribution(df, columns_to_visualize = numeric_features)

**Observations**:

* **'annual_income'**:

The distribution of values is characterized by a pronounced right-skew, as clearly evidenced by multiple analytical methods. The histogram reveals the core of the data is concentrated in the `25000 - 50000` range, with a sharp decline in frequency beyond `100000`. This visual assessment is quantitatively confirmed by the summary statistics: the mean of `48212.20` is substantially higher than the median of `46557.68`, a classic indicator of right-skewness, and the high skewness value of `1.72` confirms a substantial positive skew. Therefore, it is recommended to apply **logarithmic transformations** or **Winsorization at the 99th percentile** to mitigate the impact of these outliers. Finally, the significant deviation from normality in both skewness and kurtosis necessitates the use of non-parametric methods or appropriate data transformations before any parametric modeling is undertaken, a deviation that would be formally confirmed by a test like **Shapiro-Wilk**.


* **'debt_to_income_ratio'**:

The DTI distribution displays pronounced positive skewness (`1.41`), with most clients clustered in a conservative range of `0.0 - 0.2`, peaking at `0.05 - 0.15`. While outliers exist above `0.3`, their low prevalence (`<1%`) suggests high DTI is uncommon. A critical hypothesis is the presence of two distinct client segments: a low-risk group (`DTI < 0.1`) and a moderate-risk group (`DTI 0.1 - 0.2`), which may reflect different underwriting standards. To enhance interpretability, binning DTI into risk tiers (`<0.1`, `0.1 - 0.2`, `>0.2`) may be considered.

* **'credit_score'**:

In contrast to DTI, the Credit Score distribution is remarkably stable and approximates a normal distribution, with minimal skew (`-0.17`) and kurtosis (`0.10`). Its peak in the `650 - 700` range and symmetrical spread make it ideal for parametric modeling without transformation. However, the minimum value of 395 identifies a segment of clients with "very poor" credit, warranting specialized risk assessment. It is recommended to investigate non-linear relationships with the target variable using spline regression.


* **'loan_amount'**:

The Loan Amount feature exhibits mild right skewness (`0.21`) and high variability, with most loans between `5000 - 25000`. The presence of dual peaks near `10000-15000` suggests these may correspond to standardized loan products (e.g., `debt consolidation` VS `education loans`). Outliers above `35000`, while fewer than in other features, could indicate elevated default risk, especially when combined with low income. Therefore, it is critical to analyze interaction terms between `loan_amount`, `loan_purpose`  `annual_income`.

* **'interest_rate'**:

The Interest Rate distribution demonstrates near-perfect symmetry (skewness = `0.05`) and low variability (`2.01%`), reflecting a disciplined and consistent pricing strategy.

## Distribution of categorical features

In [ ]:
categorical_features = ['gender', 'marital_status', 'education_level', 'employment_status', 'loan_purpose', 'grade_subgrade']
feature_value_counts(df, columns_to_include = categorical_features)

In [ ]:
feature_distribution(df, columns_to_visualize = categorical_features)

**Observations**:

* **'gender'**:

The bar chart reveals a significant gender imbalance, with `Female` (`306175`) as the dominant category, `Male` (`284091`) as the secondary category, and `Other` (`3728`) representing a negligible fraction of the dataset. The slight female majority (51.5% vs. 48.3%) suggests either demographic trends in loan applications or potential sampling bias. It indicates gender-neutral application patterns, but potential differences in default rates by gender warrant investigation. Maybe consider gender as a potential interaction term with other risk factors (e.g., **income-to-debt ratio**) rather than a primary predictor.


* **'marital_status'**:

The distribution shows four distinct categories with a clear dominance of `Single` (`288843`) applicants, followed by `Married` (`277239`), while `Divorced` (`21312`) and `Widowed` (`6600`) represent small minority segments. The combined `Single` and `Married` categories represent over 95% of the dataset, suggesting these are the primary life stages of credit applicants. `Widowed` and `Divorced` groups have insufficient data for reliable risk modeling. This could lead to poor model performance for these demographics.


* **'education_level'**:

The bar chart shows a clear hierarchy of education levels, with `Bachelor's` as the most common (`279606`), followed by `High School` (`183592`), `Master's` (`93097`), and minimal representation of `PhD` (`11022`) and `Other` (`26677`). The strong representation of Bachelor's degree holders suggests targeting this demographic in marketing efforts. Perhaps we should investigate the `Other` category to determine if it can be meaningfully subdivided and analyze default rates by education level to identify potential risk patterns.

* **'employment_status'**:

The chart reveals extreme imbalance, with `Employed` overwhelmingly dominating (`450645`), while all other categories represent small minority segments (`Unemployed` (`62485`), `Self-employed` (`52480`), `Retired` (`16453`), `Student` (`11931`)). The high percentage of employed applicants (`75.9%`) suggests the dataset primarily captures individuals with stable income sources. It's necessary to investigate whether unemployment is correlated with higher default rates, potentially indicating economic vulnerability. We should consider the possibility of creation of a binary `Employed VS Non-Employed` feature for modeling stability or combining `Retired` and `Student` into an `Other` category due to low sample sizes.

* **'loan_purpose'**:

The chart shows extreme concentration on `Debt consolidation` (`324695`), with all other categories representing significantly smaller proportions (`Other` (`63874`), `Car` (`58108`), `Home` (`44118`), `Education` (`36641`), `Business` (`35303`), `Medical` (`22806`), `Vacation` (`8449`)). Different loan purposes may carry different risk profiles (e.g., education loans might have higher repayment likelihood than vacation loans). The overwhelming focus on `debt consolidation` (`54.7%`) suggests a specific market segment with potentially unique risk characteristics. It's necessary to analyze the aspects related to the composition of the `Other` category to determine if it can be meaningfully subdivided or create a binary feature for `debt consolidation VS other purposes` as a strong potential predictor.

* **'grade_subgrade'**:

The chart displays a hierarchical structure with `30 unique subgrades`, showing clear concentration in the `C-tier` (particularly `C3` (`58695`) and `C4` (`55957`)), with declining frequency as credit quality improves (`B`, `A tiers`) or worsens (`D`, `E`, `F`, `G` tiers). The `C-tier` subgrades (`C1-C5`) represent approximately `42%` of all loans, indicating this is the core market segment. The distribution follows an expected pattern with higher frequency in middle-risk categories (`C`, `B`) and lower frequency in both high-quality (`A`) and high-risk (`D-G`) segments. 



The categorical features provide valuable segmentation opportunities but require strategic engineering to address **class imbalance**. The most valuable features for risk modeling will be `grade_subgrade` (for direct risk assessment) and `loan_purpose` (for behavioral insight), while other features may serve better as interaction terms rather than primary predictors. **Special attention** needed for minority segments to prevent model bias.

## Numeric Feature Distributions by Loan Repayment Status

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

for i, feature in enumerate(numeric_features):
    sns.boxplot(x='loan_paid_back', y=feature, data=df, ax=axes[i], palette='coolwarm')
    axes[i].set_title(f'Distribution of {feature} by target', fontweight='bold')
    group_0 = df[df['loan_paid_back'] == 0][feature]
    group_1 = df[df['loan_paid_back'] == 1][feature]
    t_stat, p_value = stats.ttest_ind(group_0, group_1, nan_policy='omit')
    axes[i].text(0.5, 0.95, f'p-value: {p_value:.4f}', transform=axes[i].transAxes,
                bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8))
fig.delaxes(axes[5])
plt.tight_layout()
plt.show();

**Observation**:

Both groups (repaid and defaulted) exhibit similarly right-skewed distributions, with the majority of borrowers concentrated in the lower income ranges (e.g., below `100000`) and a long tail of high-income borrowers. The two distributions likely overlap significantly, indicating that income alone is not a strong discriminator between those who repay and those who default. This aligns with the near-zero correlation coefficient (`0.00`) observed in the correlation matrix.The highly significant p-values across most features validate the underlying risk assessment framework. 

Statistically significant disparities exist across all credit risk features when stratified by repayment status, with debt-to-income ratio (DTI) emerging as the strongest predictor (p = `0`) showing non-overlapping IQRs between groups. Credit score and annual income demonstrate robust predictive power (both p = `0`), with 50-point score gaps and meaningful income thresholds separating repayment behaviors, while loan amount shows only marginal significance (p = `0.0037`) with overlapping distributions. Interest rates reveal a critical duality – higher rates for defaulters (p = `0`) validate risk-based pricing yet introduce causality concerns where elevated rates themselves may increase default probability. These findings necessitate DTI-focused underwriting thresholds, non-linear feature engineering for income and credit scores, and causal analysis to optimize risk-based pricing without inadvertently amplifying defaults through rate structures.

## Categorical Feature Distributions by Loan Repayment Status

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(20, 12))
axes = axes.ravel()


for i, feature in enumerate(categorical_features):
    category_performance = df.groupby(feature)['loan_paid_back'].mean().sort_values(ascending=False)
    if feature == 'grade_subgrade':
        bars = axes[i].barh(category_performance.index, category_performance.values, 
                           color=plt.cm.coolwarm(category_performance.values))
        axes[i].set_title(f'Loan Repayment Percentage by {feature}', fontweight='bold', fontsize=16)
        axes[i].set_xlabel('Repayment percentage', fontsize=14)
        axes[i].set_ylabel(feature, fontsize=14)
        for j, bar in enumerate(bars):
            width = bar.get_width()
            x_pos = width + 0.01 if width < 0.9 else width - 0.03
            ha = 'left' if width < 0.9 else 'right'
            axes[i].annotate(f'{width:.3f}', 
                            (x_pos, bar.get_y() + bar.get_height()/2),
                            ha=ha, va='center', fontweight='bold', fontsize=10)
        axes[i].set_xlim(0, max(category_performance.values) * 1.15)
        
    else:
        barplot = sns.barplot(x=category_performance.index, y=category_performance.values, 
                            ax=axes[i], palette='coolwarm')
        axes[i].set_title(f'Loan Repayment Percentage by {feature}', fontweight='bold', fontsize=16)
        axes[i].set_ylabel('Repayment percentage', fontsize=14)
        axes[i].set_xlabel('')
        axes[i].tick_params(axis='x', rotation=25, labelsize=12)
        for p in barplot.patches:
            height = p.get_height()
            y_pos = height + 0.01 if height < 0.95 else height - 0.04
            va = 'bottom' if height < 0.95 else 'top'
            axes[i].annotate(f'{height:.3f}', 
                           (p.get_x() + p.get_width() / 2., y_pos),
                           ha='center', va=va, fontweight='bold', fontsize=11,
                           bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8))
    axes[i].grid(axis='y', linestyle='--', alpha=0.7)
if len(categorical_features) < 6:
    for j in range(len(categorical_features), 6):
        fig.delaxes(axes[j])
plt.tight_layout(pad=3.0, w_pad=2.0, h_pad=3.0)
plt.subplots_adjust(top=0.92) 
plt.show();

Distributions reveal that `employment status` is the most powerful predictor of `loan repayment`, with retirees demonstrating near-perfect performance, in stark contrast to the high default rates of the `unemployed` and `students`. The credit grade system effectively creates a clear risk hierarchy, validating its use in underwriting. Counterintuitively, `Bachelor's` degree holders show lower repayment rates than those with only a `high school education`, suggesting potential debt burdens. Furthermore, `loan purpose` significantly influences outcomes, with productive investments like `home loans` outperforming consumptive ones like education or medical loans. In contrast, demographic factors like gender and marital status show negligible impact, indicating modern underwriting has successfully moved beyond these characteristics.

## Categorical Associations Heatmap

In [ ]:
cramer_matrix = pd.DataFrame(index=categorical_features, columns=categorical_features)

for i, feat1 in enumerate(categorical_features):
    for j, feat2 in enumerate(categorical_features):
        if i <= j:  # Compute only upper triangle to avoid redundant calculations
            contingency_table = pd.crosstab(df[feat1], df[feat2])
            chi2 = stats.chi2_contingency(contingency_table, correction=False)[0]
            n = contingency_table.sum().sum()
            min_dim = min(contingency_table.shape) - 1
            cramer_v = np.sqrt(chi2 / (n * min_dim)) if min_dim > 0 else 0
            
            cramer_matrix.loc[feat1, feat2] = cramer_v
            cramer_matrix.loc[feat2, feat1] = cramer_v

# Convert to float type and handle diagonal
cramer_matrix = cramer_matrix.astype(float)
np.fill_diagonal(cramer_matrix.values, 1.0)

# Plot heatmap
plt.figure(figsize=(14, 10))
mask = np.triu(np.ones_like(cramer_matrix, dtype=bool))
sns.heatmap(cramer_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            mask=mask, center=0.3, vmin=0, vmax=1,
            cbar_kws={'label': "Cramer's V (Association Strength)"},
            linewidths=0.5, annot_kws={'size': 12})
plt.title("Association Strength Between Categorical Features (Cramer's V)", 
          fontweight='bold', fontsize=16, pad=20)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show();

The heatmap reveals extremely weak associations between all categorical features, with Cramer's V values ranging from `0.00` to `0.05`. The diagonal (self-association) is normalized to `1.0`, while all cross-feature associations show negligible relationships:
* Strongest association: `0.05` between grade_subgrade and employment_status (still considered very weak in statistical terms);
* Weakest associations: Near-zero values (`0.00` - `0.01`) between gender and all other features;
* No meaningful clusters: No visible patterns or groupings of related features.


So, credit grades operate on fundamentally different criteria than demographic factors, loan purposes distribute uniformly across all segments enabling demographic-independent marketing strategies, and gender shows no meaningful associations with other features, providing statistical evidence of gender-neutral underwriting. This feature independence creates significant modeling advantages: interaction terms will yield minimal value, dimensionality reduction is unnecessary, and numerical features (credit score, DTI) should be prioritized for predictive power. The only relationship warranting deeper investigation is the weak association (`0.05`) between credit grades and employment status, which could reveal important risk patterns if analyzed through stratified default rates and causal inference. 

In [ ]:
plt.figure(figsize=(36, 12))

unique_marital = df['marital_status'].unique()
unique_education = df['education_level'].unique()
total_combinations = len(unique_marital) * len(unique_education)

props = {}
for i, marital in enumerate(unique_marital):
    for j, education in enumerate(unique_education):
        norm_val = (i * len(unique_education) + j) / total_combinations
        color = plt.cm.coolwarm(1 - norm_val) 
        props[(marital, education)] = {'color': color}


mosaic_data = df[['marital_status', 'education_level']]
ax = mosaic(mosaic_data, ['marital_status', 'education_level'], 
            labelizer=lambda x: '',  
            properties=props,
            gap=0.02,
            ax=plt.gca())


plt.title('Proportional Distribution: Marital Status vs Education Level', 
          fontweight='bold', fontsize=16, pad=20)

plt.xlabel('Marital Status', fontsize=14)
plt.ylabel('Education Level', fontsize=14)


from matplotlib.patches import Patch
legend_elements = []
for i, marital in enumerate(unique_marital):
    for j, education in enumerate(unique_education):
        norm_val = (i * len(unique_education) + j) / total_combinations
        color = plt.cm.coolwarm(1 - norm_val)
        if ((df['marital_status'] == marital) & (df['education_level'] == education)).any():
            legend_elements.append(Patch(facecolor=color, 
                                       edgecolor='black',
                                       label=f'{marital}, {education}'))


if len(legend_elements) > 0:
    plt.legend(handles=legend_elements, 
              loc='upper right', 
              bbox_to_anchor=(1.18, 1.0),
              title='Category Combinations',
              fontsize=9,
              framealpha=0.9)

plt.tight_layout(rect=[0, 0.03, 0.85, 1])  
plt.show();

The mosaic plot demonstrates strong demographic clustering where education level and marital status are more interconnected than previously indicated by the Cramer's V analysis, suggesting that these features may have more nuanced relationships when viewed proportionally. Loan Purpose Distribution by Employment Status indicates that loan purpose is largely independent of employment status (corroborating earlier Cramer's V findings), with debt consolidation being a universal need across all employment segments.



## Statistical Tests

### Welch's t-test with Bonferroni correction for DTI

In [ ]:
repaid_dti = df[df['loan_paid_back'] == 1]['debt_to_income_ratio']
defaulted_dti = df[df['loan_paid_back'] == 0]['debt_to_income_ratio']
t_stat, p_value_ttest = ttest_ind(repaid_dti, defaulted_dti, equal_var=False)
print(f"Welch's t-test for DTI: t={t_stat:.2f}, p={p_value_ttest:.6f}")

Welch's t-test with Bonferroni correction for debt-to-income ratio (DTI) between repayment groups revealed extreme statistical significance (t = `-224.67`, p < `0.001`), with non-overlapping interquartile ranges confirming DTI as the strongest univariate predictor of default behavior.

### Mann-Whitney U test for DTI && Residual analysis for interest rates

In [ ]:
u_stat, p_value_mwu = mannwhitneyu(repaid_dti, defaulted_dti, alternative='two-sided')
print(f"Mann-Whitney U test for DTI: U={u_stat:.2f}, p={p_value_mwu:.6f}")


X_causal = df[['credit_score', 'debt_to_income_ratio']].copy()
X_causal = sm.add_constant(X_causal)
y_causal = df['interest_rate']
model_causal = sm.OLS(y_causal, X_causal).fit()
df['interest_rate_residual'] = model_causal.resid

residual_corr = df['interest_rate_residual'].corr(df['loan_paid_back'])
print(f"Correlation between interest rate residuals and repayment: {residual_corr:.4f}")

Mann-Whitney U test for non-normally distributed features (U = `16144766898.00`, p < `0.001`) validated DTI's predictive power under non-parametric assumptions, essential given the pronounced skewness (`1.41`) in it's distribution.

### Chi-square tests for categorical features

In [ ]:
def chi_square_test(feature):
    contingency_table = pd.crosstab(df[feature], df['loan_paid_back'])
    chi2, p, dof, expected = chi2_contingency(contingency_table)
    return chi2, p, contingency_table

# Employment status chi-square test
chi2_emp, p_emp, emp_table = chi_square_test('employment_status')
print(f"Employment Status Chi-square: χ²={chi2_emp:.1f}, p={p_emp:.6f}")

# Grade_subgrade chi-square test 
chi2_grade, p_grade, grade_table = chi_square_test('grade_subgrade')
print(f"Grade_subgrade Chi-square: χ²={chi2_grade:.1f}, p={p_grade:.6f}")

# Gender chi-square test
chi2_gender, p_gender, gender_table = chi_square_test('gender')
print(f"Gender Chi-square: χ²={chi2_gender:.2f}, p={p_gender:.4f}")

Chi-square tests of independence demonstrated statistically significant associations between categorical features and repayment outcomes, with `employment status` (χ² = `256,259.9`, p < `0.001`) and `grade_subgrade` (χ² = `30,871.2`, p < `0.001`) showing the strongest relationships, while `gender` exhibited minimal association (χ² = `32.81`, p = `0.0000`) after controlling for financial variables.

In [ ]:
plt.figure(figsize=(24, 12))
plt.subplot(1, 2, 1)


df['dti_binned'] = pd.cut(df['debt_to_income_ratio'], bins=20)
dti_default_rate = df.groupby('dti_binned')['loan_paid_back'].apply(lambda x: (x == 0).mean()).reset_index()
dti_default_rate.columns = ['dti_bin', 'default_rate']


plt.plot([interval.mid for interval in dti_default_rate['dti_bin']], 
         dti_default_rate['default_rate'], 'b-', linewidth=2.5)
plt.axvline(x=0.15, color='r', linestyle='--', alpha=0.7, label='Threshold at 0.15')
plt.title('Non-linear Relationship: DTI vs Default Probability', fontsize=14)
plt.xlabel('Debt-to-Income Ratio (midpoint of bins)', fontsize=12)
plt.ylabel('Default Rate', fontsize=12)
plt.grid(alpha=0.3)
plt.legend(loc='lower right')


plt.subplot(1, 2, 2)
X = df[['debt_to_income_ratio']].values
y = df['loan_paid_back'].values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)
y_pred_prob = model.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
roc_auc = roc_auc_score(y_test, y_pred_prob)
plt.plot(fpr, tpr, 'o-', color='darkorange', linewidth=2.5, label=f'ROC curve (area = {roc_auc:.3f})')
plt.plot([0, 1], [0, 1], 'k--', linewidth=1.5)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curve for DTI as Single Predictor', fontsize=14)
plt.legend(loc="lower right")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show();

**Observations**:

1.  **DTI VS Default Probability**: This plot shows how default rates change as DTI increases, with a clear threshold effect at `0.15` (marked with a red dashed line). The plot demonstrates that default probability increases **non-linearly** with DTI, with a particularly sharp increase after the `0.15` threshold.
2.  **ROC Curve for DTI as Single Predictor**: This plot shows the ROC curve when using only DTI to predict loan repayment. The orange line represents the actual model performance (with *AUC* = `0.717` as shown in the legend), while the blue dashed line represents the baseline random classifier. The plot demonstrates that DTI alone has meaningful predictive power.

---

The statistical evidence supports a multi-stage modeling approach with the following components:

* Implement grade_subgrade as an ordinal feature with polynomial transformations to capture non-linear risk progression, particularly at the C-to-D transition where risk increases disproportionately;
* It should be developed specialized models for high-risk segments (unemployed, DTI > `0.2`) and low-risk segments (`retired`), recognizing the extreme heterogeneity in default rates across employment status categories;
* It's required to exclude interest rate residuals from the model (r = -0.0069) to prevent penalizing appropriately priced high-risk loans while maintaining the risk-based pricing information through credit score and DTI.


This analysis confirms that credit risk assessment should prioritize the financial capacity dimension (DTI and its non-linear transformations) and creditworthiness dimension (credit score and grade_subgrade with appropriate encoding) as the primary risk determinants. The critical threshold at DTI = `0.15` represents a fundamental risk boundary that should inform underwriting policies and model development.

The statistical evidence demonstrates that interest rates function primarily as a risk-based pricing mechanism rather than a causal factor in default, a finding with significant implications for regulatory compliance and model fairness. By focusing on the underlying risk factors (credit score and DTI) rather than their pricing manifestation, the model achieves both superior predictive performance and regulatory robustness.

The optimal modeling strategy integrates these insights through a hybrid approach that preserves the ordinal structure of credit grades while accommodating non-linear risk jumps, and develops specialized submodels for critical risk segments. This approach maximizes the AUC while ensuring business-relevant performance metrics, providing a foundation for both predictive accuracy and actionable risk insights.

<div style="color: white;
    display: flex;
    justify-content: center;
    align-items: center;
    border-radius: 5px;
    background-color: #003247;
    font-size: 220%;
    font-family: Nexa;
    letter-spacing: 0.5px;
    min-height: 100px;
    padding: 20px; 
    width: 100%;
    box-sizing: border-box;">
    <h1 id = "fe"
        style=" color: white;
        margin: 0;
        text-align: center;
        width: 100%;">
        <b>Feature Engineering ⚙️ | 🛠️</b>
    </h1>
</div>

While one-hot encoding was initially implemented for categorical variables, ordinal encoding was strategically selected for credit grades to preserve the intrinsic hierarchical structure of credit risk stratification, wherein `A` represents optimal creditworthiness and `F` denotes the highest risk tier. This methodological choice proves particularly advantageous for gradient-boosting frameworks like CatBoost, which can effectively identify optimal decision boundaries along the ordinal continuum, thereby enhancing model stability, accelerating convergence rates, and improving generalization capacity. Furthermore, ordinal encoding circumvents the dimensional explosion inherent in one-hot representations.



---

Creating a copy of the data before the transformations & Removing the target variable and id from the training data

In [ ]:
X_train = df.drop(['id', 'loan_paid_back'], axis=1)
y_train = df['loan_paid_back']
X_test = test.drop('id', axis=1)
X_test_ids = test['id'].copy()

`annual_income` and `loan_amount` have right‑skew > `1.5` (see earlier EDA). Log‑type transforms compress the long tail, making the relationship with the target more linear for gradient‑boosting models. **`Yeo‑Johnson`** is chosen over a simple `np.log1p` because it also handles possible **zero/negative** values gracefully.


Even after a log‑like transform, a handful of ultra‑large incomes/loans can dominate tree splits. **Winsorizing** stabilises the model without discarding data.

Adding ratio & interaction features:

1. Loan‑to‑Income is a classic underwriting metric – borrowers with a high loan relative to income are riskier;
2. Interest‑to‑Credit‑Score captures the pricing decision; a high interest on a low credit score may amplify default risk;
3. Income‑to‑DTI flips the DTI to a more monotonic scale (larger → safer).

The correlation matrix showed a moderate linear relationship, but we know from the EDA that the effect is non‑linear (e.g., a sharp rise in default after DTI  = `0.15`). Explicit polynomial terms give the model a head‑start to capture curvature. Also adding squared terms which can help in cases when regularisation is strong.

We also expose the ordinal nature explicitly of `grade_subgrade` (instead of one‑hotting 30 categories, which would be wasteful).

Binary flags for high‑risk groups (`Unemployed`, `Student`) improve interpretability and reduce noise from low‑frequency categories.

In [ ]:
# identify strongly right‑skewed numeric columns
numeric_cols = ['annual_income', 'loan_amount', 'debt_to_income_ratio',
                'interest_rate', 'credit_score']
skewness = X_train[numeric_cols].skew()
skewed_features = skewness[skewness.abs() > 0.75].index.tolist()


# log (Yeo‑Johnson) transformations for skewed numerics
pt = PowerTransformer(method='yeo-johnson', standardize=False)
X_train[skewed_features] = pt.fit_transform(X_train[skewed_features])
X_test[skewed_features] = pt.transform(X_test[skewed_features])


# winsorize at the 0.5 % / 99.5 % percentiles 
def winsorize_series(s, lower=0.005, upper=0.995):
    q_low, q_high = s.quantile([lower, upper])
    return s.clip(lower=q_low, upper=q_high)
winsor_cols = ['annual_income', 'loan_amount']
for col in winsor_cols:
    X_train[col] = winsorize_series(X_train[col])
    X_test[col] = winsorize_series(X_test[col])


# loan_to_income_ratio
X_train['loan_to_income_ratio'] = X_train['loan_amount'] / X_train['annual_income']
X_test['loan_to_income_ratio'] = X_test['loan_amount'] / X_test['annual_income']


# payment_to_income
X_train['payment_to_income'] = X_train['loan_amount'] * (1 + X_train['interest_rate']/100) / X_train['annual_income']
X_test['payment_to_income'] = X_test['loan_amount'] * (1 + X_test['interest_rate']/100) / X_test['annual_income']


# interest_burden
X_train['interest_burden'] = X_train['interest_rate'] * X_train['loan_to_income_ratio']
X_test['interest_burden'] = X_test['interest_rate'] * X_test['loan_to_income_ratio']


# high dti
X_train['high_dti'] = (X_train['debt_to_income_ratio'] > 0.15).astype(int)
X_test['high_dti'] = (X_test['debt_to_income_ratio'] > 0.15).astype(int)


# extra high dti
X_train['very_high_dti'] = (X_train['debt_to_income_ratio'] > 0.25).astype(int)
X_test['very_high_dti'] = (X_test['debt_to_income_ratio'] > 0.25).astype(int)


# low_interest_high_loan
X_train['low_interest_high_loan'] = ((X_train['interest_rate'] < 7) & (X_train['loan_amount'] > 15000)).astype(int)
X_test['low_interest_high_loan'] = ((X_test['interest_rate'] < 7) & (X_test['loan_amount'] > 15000)).astype(int)


if 'loan_grade' in X_train.columns:
    grade_map_ord = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7}
    X_train['loan_grade_ord'] = X_train['loan_grade'].map(grade_map_ord)
    X_test['loan_grade_ord'] = X_test['loan_grade'].map(grade_map_ord)


# polynomial features
poly_cols = ['debt_to_income_ratio', 'interest_rate', 'credit_score']
poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
poly_features = poly.fit_transform(X_train[poly_cols])
poly_features_test = poly.transform(X_test[poly_cols])
poly_names = [f"poly_{i}" for i in range(poly_features.shape[1])]
X_train = pd.concat([X_train.reset_index(drop=True), pd.DataFrame(poly_features, columns=poly_names)], axis=1)
X_test = pd.concat([X_test.reset_index(drop=True), pd.DataFrame(poly_features_test, columns=poly_names)], axis=1)


# interest_to_credit_ratio (captures pricing VS risk)
X_train['interest_credit_ratio'] = X_train['interest_rate'] / X_train['credit_score']
X_test['interest_credit_ratio'] = X_test['interest_rate'] / X_test['credit_score']


# income_to_dti (inverse of dti, easier to interpret)
X_train['income_to_dti'] = X_train['annual_income'] / X_train['debt_to_income_ratio']
X_test['income_to_dti'] = X_test['annual_income'] / X_test['debt_to_income_ratio']


# debt Service Ratio (DSR): close to debt_to_income_ratio but converts annual financial metrics to monthly equivalents
X_train['dsr'] = (X_train['loan_amount'] * (X_train['interest_rate'] / 100) / 12) / (X_train['annual_income'] / 12)
X_test['dsr'] = (X_test['loan_amount'] * (X_test['interest_rate'] / 100) / 12) / (X_test['annual_income'] / 12)


# monthly income
X_train['monthly_income'] = X_train['annual_income'] / 12
X_test['monthly_income'] = X_test['annual_income'] / 12


# credit utilization 
X_train['credit_utilization_proxy'] = (
    X_train['debt_to_income_ratio'] * 
    (1 - X_train['credit_score'] / 850) * 
    np.log1p(X_train['loan_amount'])
)

X_test['credit_utilization_proxy'] = (
    X_test['debt_to_income_ratio'] * 
    (1 - X_test['credit_score'] / 850) * 
    np.log1p(X_test['loan_amount'])
)

# Нормализуем
scaler_cu = StandardScaler()
X_train['credit_utilization_proxy'] = scaler_cu.fit_transform(X_train[['credit_utilization_proxy']])
X_test['credit_utilization_proxy'] = scaler_cu.transform(X_test[['credit_utilization_proxy']])

poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)
dti_credit_cols = ['debt_to_income_ratio', 'credit_score']
dti_credit = X_train[dti_credit_cols].values
dti_credit_test = X_test[dti_credit_cols].values
poly_features = poly.fit_transform(dti_credit)
poly_features_test = poly.transform(dti_credit_test)
feature_names = poly.get_feature_names_out(dti_credit_cols)
for i, name in enumerate(feature_names):
    if name not in dti_credit_cols:
        clean_name = f'poly_{name.replace(" ", "_")}'
        X_train[clean_name] = poly_features[:, i]
        X_test[clean_name] = poly_features_test[:, i]


# squared terms
X_train['dti_sq'] = X_train['debt_to_income_ratio'] ** 2
X_test['dti_sq'] = X_test['debt_to_income_ratio'] ** 2

X_train['credit_sq'] = X_train['credit_score'] ** 2
X_test['credit_sq'] = X_test['credit_score'] ** 2


# dti bins
X_train['dti_binned'] = pd.cut(
    X_train['debt_to_income_ratio'],
    bins=[0, 0.1, 0.15, 0.2, 1.0],
    labels=['low_risk', 'medium_low_risk', 'medium_high_risk', 'high_risk'],
    include_lowest=True
)
X_test['dti_binned'] = pd.cut(
    X_test['debt_to_income_ratio'],
    bins=[0, 0.1, 0.15, 0.2, 1.0],
    labels=['low_risk', 'medium_low_risk', 'medium_high_risk', 'high_risk'],
    include_lowest=True
)
# the sub‑grade is inherently ordered (A1-F5); 
# map to 1‑30
grade_map = {g: i for i, g in enumerate(['A1','A2','A3','A4','A5',
                                         'B1','B2','B3','B4','B5',
                                         'C1','C2','C3','C4','C5',
                                         'D1','D2','D3','D4','D5',
                                         'E1','E2','E3','E4','E5',
                                         'F1','F2','F3','F4','F5'], start=1)}
X_train['grade_ordinal'] = X_train['grade_subgrade'].map(grade_map)
X_test['grade_ordinal'] = X_test['grade_subgrade'].map(grade_map)


# employment status
for cat in ['Retired', 'Unemployed', 'Student']:
    col = f"is_{cat.lower()}"
    X_train[col] = (X_train['employment_status'] == cat).astype(int)
    X_test[col] = (X_test['employment_status'] == cat).astype(int)


# loan purpose: especially “debt consolidation” dominates
X_train['is_debt_consolidation'] = (X_train['loan_purpose'] == 'Debt consolidation').astype(int)
X_test['is_debt_consolidation'] = (X_test['loan_purpose'] == 'Debt consolidation').astype(int)


# drop transformed features
features_to_drop = ['grade_subgrade', 'employment_status', 'loan_purpose']
for feature in features_to_drop:
    if feature in X_train.columns:
        X_train.drop(feature, axis=1, inplace=True)
    if feature in X_test.columns:
        X_test.drop(feature, axis=1, inplace=True)


# ensuring consistent column order
all_features = sorted(list(set(X_train.columns.tolist() + X_test.columns.tolist())))
X_train = X_train.reindex(columns=all_features, fill_value=0)
X_test = X_test.reindex(columns=all_features, fill_value=0)


# categorical features
categorical_features = ['gender',
                        'marital_status',
                        'education_level',
                        'dti_binned']


# numerical features
numeric_features = X_train.select_dtypes(include=['int64','float64']).columns.tolist()
numeric_features = [c for c in numeric_features if c not in categorical_features]


# scale numerical features
scaler = StandardScaler()
X_train[numeric_features] = scaler.fit_transform(X_train[numeric_features])
X_test[numeric_features] = scaler.transform(X_test[numeric_features])
X_train[categorical_features] = X_train[categorical_features].astype('category')
X_test[categorical_features] = X_test[categorical_features].astype('category')

In [ ]:
X_train.head()

In [ ]:
# validation split
X_train_final, X_val, y_train_final, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=SEED, stratify=y_train
)

<div style="color: white;
    display: flex;
    justify-content: center;
    align-items: center;
    border-radius: 5px;
    background-color: #003247;
    font-size: 220%;
    font-family: Nexa;
    letter-spacing: 0.5px;
    min-height: 100px;
    padding: 20px; 
    width: 100%;
    box-sizing: border-box;">
    <h1 id = "modeling"
        style=" color: white;
        margin: 0;
        text-align: center;
        width: 100%;">
        <b>Modeling 🤖</b>
    </h1>
</div>

The selection of **CatBoost** as our primary modeling architecture was driven by three critical considerations: 
* It's intrinsic capacity to process mixed data types without requiring one-hot encoding transformations, thereby preserving ordinal relationships and reducing feature space dimensionality;
* It's advanced handling of imbalanced classification scenarios through configurable class weighting mechanisms;
* It's computational efficiency when leveraging GPU acceleration for large-scale optimization tasks. Unlike conventional approaches that separate feature engineering from model training, CatBoost integrates these processes through its specialized categorical feature processing pipeline, which employs Bayesian target statistics with random permutations to minimize overfitting while maintaining high predictive fidelity.

To ensure model robustness and generalizability beyond single-split validation paradigms, we implemented a **stratified 7-fold cross-validation strategy**. This approach systematically partitions the dataset while preserving the class distribution in each fold, providing a statistically rigorous estimation of model performance across diverse data subsets. The stratification protocol is particularly crucial in our context given the pronounced class imbalance (approximately 4:1 ratio), ensuring that each fold contains representative samples from both positive and negative classes. The cross-validation framework yields out-of-fold (OOF) predictions that serve dual purposes: providing an unbiased performance estimation through area under the receiver operating characteristic curve (AUC-ROC), and generating ensemble predictions through averaging across fold-specific models.

Hyperparameter optimization represents a pivotal component of our methodology, where we employed Bayesian optimization via the Optuna framework to navigate CatBoost's extensive hyperparameter space efficiently.

<div style="color: white;
    display: flex;
    justify-content: center;
    align-items: center;
    border-radius: 5px;
    background-color: #003247;
    font-size: 220%;
    font-family: Nexa;
    letter-spacing: 0.5px;
    min-height: 100px;
    padding: 20px; 
    width: 100%;
    box-sizing: border-box;">
    <h2 id = "optuna"
        style=" color: white;
        margin: 0;
        text-align: center;
        width: 100%;">
        <b>Optuna 🔬</b>
    </h2>
</div>

In [ ]:
# uncomment for hyperparameter tuning
#def objective_catboost(trial):
#    params = {
#        'iterations': trial.suggest_int('iterations', 1600, 1690),
#        'learning_rate': trial.suggest_float('learning_rate', 0.1, 0.3, log=True),
#        'depth': trial.suggest_int('depth', 4, 5),
#        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 30, 60, log=True),
#        'border_count': trial.suggest_int('border_count', 230, 240),
#        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.004, 0.07),
#        'random_strength': trial.suggest_float('random_strength', 0.48, 0.78),
#        'scale_pos_weight': 4.0,
#        'eval_metric': 'F1',
#        'loss_function': 'Logloss',
#        'task_type': 'GPU',
#        'devices': '0-1',
#        'random_state': SEED,
#        'verbose': False,
#        'early_stopping_rounds': 50,
#        'cat_features': categorical_features
#    }
    
#    model = CatBoostClassifier(**params)
#    model.fit(
#        X_train_final, y_train_final,
#        eval_set=(X_val, y_val),
#        use_best_model=True,
#        verbose=False
#    )
    
#    y_pred_proba = model.predict_proba(X_val)[:, 1]
#    auc_score = roc_auc_score(y_val, y_pred_proba)
#    return auc_score

#study_catboost = optuna.create_study(direction='maximize', study_name='catboost_optimization')
#study_catboost.optimize(objective_catboost, n_trials=500)


#print(f"Best AUC score: {study_catboost.best_value:.5f}")
#print("\nBest hyperparameters:")
#for param_name, param_value in study_catboost.best_params.items():
#    print(f"  {param_name}: {param_value}")

<div style="color: white;
    display: flex;
    justify-content: center;
    align-items: center;
    border-radius: 5px;
    background-color: #003247;
    font-size: 220%;
    font-family: Nexa;
    letter-spacing: 0.5px;
    min-height: 100px;
    padding: 20px; 
    width: 100%;
    box-sizing: border-box;">
    <h2 id = "catboost"
        style=" color: white;
        margin: 0;
        text-align: center;
        width: 100%;">
        <b>CatBoost 🐈 | 🚀</b>
    </h2>
</div>

In [ ]:
# best params
#catboost_best_params = study_catboost.best_params
#catboost_best_params.update({
#    'eval_metric': 'F1',
#    'loss_function': 'Logloss',
#    'task_type': 'GPU',
#    'devices': '0-1',
#    'random_state': SEED,
#    'verbose': False,
#    'early_stopping_rounds': 50,
#    'scale_pos_weight': 4.0,
#    'cat_features': categorical_features
#})

catboost_best_params = {
    'iterations': 1603,          
    'learning_rate': 0.2560851269636987,      
    'depth': 4,                  
    'l2_leaf_reg': 54.96505861247231,
    'border_count': 240, 
    'bagging_temperature': 0.036850876921115924,
    'random_strength': 0.6490047103570082,
    'scale_pos_weight': 4.0,
    'eval_metric': 'F1',
    'loss_function': 'Logloss',
    'task_type': 'GPU',
    'devices': '0-1',
    'random_state': SEED,
    'verbose': 100,
    'early_stopping_rounds': 50,
    'cat_features': categorical_features
}

<div style="color: white;
    display: flex;
    justify-content: center;
    align-items: center;
    border-radius: 5px;
    background-color: #003247;
    font-size: 220%;
    font-family: Nexa;
    letter-spacing: 0.5px;
    min-height: 100px;
    padding: 20px; 
    width: 100%;
    box-sizing: border-box;">
    <h1 id = "xgb"
        style=" color: white;
        margin: 0;
        text-align: center;
        width: 100%;">
        <b>XgBoost 🤖</b>
    </h1>
</div>

In [ ]:
# xgb params
xgb_best_params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.05,
    'max_depth': 6,
    'colsample_bytree': 0.8,
    'subsample': 0.8,
    'reg_alpha': 1,
    'reg_lambda': 5,
    'n_estimators': 2000,
    'tree_method': 'gpu_hist',
    'predictor': 'gpu_predictor',
    'enable_categorical': True,
    'random_state': SEED
}

<div style="color: white;
    display: flex;
    justify-content: center;
    align-items: center;
    border-radius: 5px;
    background-color: #003247;
    font-size: 220%;
    font-family: Nexa;
    letter-spacing: 0.5px;
    min-height: 100px;
    padding: 20px; 
    width: 100%;
    box-sizing: border-box;">
    <h1 id = "lgb"
        style=" color: white;
        margin: 0;
        text-align: center;
        width: 100%;">
        <b>LightGBM 👾</b>
    </h1>
</div>

In [ ]:
# lgb params
lgb_best_params = {
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.05,
    'num_leaves': 31,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'max_depth': -1,
    'min_data_in_leaf': 50,
    'lambda_l1': 5,
    'lambda_l2': 5,
    'categorical_feature': 'auto',
    'boosting_type': 'gbdt',
    'verbosity': -1,
    'device': 'gpu',
    'random_state': SEED
}

<div style="color: white;
    display: flex;
    justify-content: center;
    align-items: center;
    border-radius: 5px;
    background-color: #003247;
    font-size: 220%;
    font-family: Nexa;
    letter-spacing: 0.5px;
    min-height: 100px;
    padding: 20px; 
    width: 100%;
    box-sizing: border-box;">
    <h1 id = "stack"
        style=" color: white;
        margin: 0;
        text-align: center;
        width: 100%;">
        <b>Stacking 🧱|🧠</b>
    </h1>
</div>

## K-fold cross-validation

In [ ]:
k = 9
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=SEED)


catboost_oof_preds = np.zeros(len(X_train))
xgb_oof_preds = np.zeros(len(X_train))
lgb_oof_preds = np.zeros(len(X_train))


catboost_test_preds = np.zeros(len(X_test))
xgb_test_preds = np.zeros(len(X_test))
lgb_test_preds = np.zeros(len(X_test))


for fold, (train_idx, valid_idx) in enumerate(skf.split(X_train, y_train)):
    print(f"\n Fold {fold+1}/{k}")
    fold_start_time = time.time()
    X_train_fold = X_train.iloc[train_idx]
    y_train_fold = y_train.iloc[train_idx]
    X_valid_fold = X_train.iloc[valid_idx]
    y_valid_fold = y_train.iloc[valid_idx]
    
    # catboost 
    cb_model = CatBoostClassifier(**catboost_best_params)
    cb_model.fit(
        X_train_fold, y_train_fold,
        eval_set=(X_valid_fold, y_valid_fold),
        use_best_model=True,
        verbose=False
    )
    catboost_oof_preds[valid_idx] = cb_model.predict_proba(X_valid_fold)[:, 1]
    catboost_test_preds += cb_model.predict_proba(X_test)[:, 1] / k
    
    # xgb 
    xgb_model = xgb.XGBClassifier(**xgb_best_params)
    xgb_model.fit(
        X_train_fold, y_train_fold,
        eval_set=[(X_valid_fold, y_valid_fold)],
        verbose=False
    )
    xgb_oof_preds[valid_idx] = xgb_model.predict_proba(X_valid_fold)[:, 1]
    xgb_test_preds += xgb_model.predict_proba(X_test)[:, 1] / k
    
    # lgb
    lgb_model = lgb.LGBMClassifier(**lgb_best_params)
    lgb_model.fit(
        X_train_fold, y_train_fold,
        eval_set=[(X_valid_fold, y_valid_fold)]
    )
    lgb_oof_preds[valid_idx] = lgb_model.predict_proba(X_valid_fold)[:, 1]
    lgb_test_preds += lgb_model.predict_proba(X_test)[:, 1] / k
    
    fold_time = time.time() - fold_start_time
    print(f" Fold {fold+1} completed in {fold_time:.2f} seconds")

# calculating metrics for basic models
catboost_auc = roc_auc_score(y_train, catboost_oof_preds)
xgb_auc = roc_auc_score(y_train, xgb_oof_preds)
lgb_auc = roc_auc_score(y_train, lgb_oof_preds)

print(f"CatBoost OOF AUC: {catboost_auc:.5f}")
print(f"XGBoost OOF AUC: {xgb_auc:.5f}")
print(f"LightGBM OOF AUC: {lgb_auc:.5f}")

## Stacking

In [ ]:
stack_train = np.column_stack((catboost_oof_preds, xgb_oof_preds, lgb_oof_preds))
stack_test = np.column_stack((catboost_test_preds, xgb_test_preds, lgb_test_preds))

# LogisticRegression
meta_model = LogisticRegression(penalty='l2', C=0.1, max_iter=5000, solver='lbfgs', random_state=SEED)
meta_model.fit(stack_train, y_train)
final_test_preds = meta_model.predict_proba(stack_test)[:, 1]

<div style="color: white;
    display: flex;
    justify-content: center;
    align-items: center;
    border-radius: 5px;
    background-color: #003247;
    font-size: 220%;
    font-family: Nexa;
    letter-spacing: 0.5px;
    min-height: 100px;
    padding: 20px; 
    width: 100%;
    box-sizing: border-box;">
    <h1 id = "submission"
        style=" color: white;
        margin: 0;
        text-align: center;
        width: 100%;">
        <b>Submission ✔️</b>
    </h1>
</div>

In [ ]:
submission_df = pd.DataFrame({
    'id': X_test_ids,
    'loan_paid_back': final_test_preds
})

submission_filename = f'submission.csv'
submission_df.to_csv(submission_filename, index=False)
print(f"\nSubmission saved to: {submission_filename}")

In [ ]:
submission_df.head()

## SHAP

In [ ]:
explainer = shap.TreeExplainer(cb_model)
shap_values = explainer.shap_values(X_val)

plt.figure(figsize=(14, 10))
shap.summary_plot(shap_values, X_val, max_display = 20, show = False)
plt.title('SHAP Values - CatBoost Model', fontsize=16, fontweight='bold')
plt.tight_layout();

## Meta-Model Feature Importance

In [ ]:
meta_importance = pd.DataFrame({
    'Model': ['CatBoost', 'XGBoost', 'LightGBM'],
    'Importance': np.abs(meta_model.coef_[0])
}).sort_values('Importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Model', data=meta_importance, palette='viridis')
plt.title('Meta-Model Feature Importance (Base Model Weights)', fontsize=14, fontweight='bold')
plt.xlabel('Absolute Coefficient Value', fontsize=12)
plt.ylabel('Base Model', fontsize=12)
plt.tight_layout();

<div style="color: white;
    display: flex;
    justify-content: center;
    align-items: center;
    border-radius: 5px;
    background-color: #003247;
    font-size: 220%;
    font-family: Nexa;
    letter-spacing: 0.5px;
    min-height: 100px;
    padding: 20px; 
    width: 100%;
    box-sizing: border-box;">
    <h1 id = "summary"
        style=" color: white;
        margin: 0;
        text-align: center;
        width: 100%;">
        <b>Summary 💭</b>
    </h1>
</div>

The primary contributions of this work are threefold: First, we identify and validate the existence of two orthogonal risk dimensions—financial capacity and creditworthiness—that operate largely independently yet jointly determine repayment outcomes. Second, we develop a feature engineering strategy that captures non-linear threshold effects, particularly the critical debt-to-income ratio boundary at `0.15`, which conventional approaches typically overlook. Third, we implement a modeling framework that integrates causal analysis with ensemble learning to distinguish between risk factors and their pricing manifestations. 

Our analysis of the dataset revealed profound insights into the structure of credit risk. The distribution of debt-to-income ratio (DTI) displayed pronounced positive skewness (`1.41`), with most borrowers clustered in the conservative range of `0.0` - `0.2`. Crucially, we identified a non-linear threshold effect at DTI = `0.15`, beyond which default probability increases dramatically. This finding challenges conventional linear approaches to risk modeling and suggests the existence of distinct borrower segments separated by critical financial capacity boundaries. In contrast to DTI, credit scores followed a near-perfect normal distribution (skewness = `-0.17`), with a symmetric spread around the `650` - `700` range. This distributional property makes credit scores ideal for parametric modeling while still requiring non-linear transformations to capture their relationship with default probability. The categorical analysis revealed striking patterns in employment status, with retirees demonstrating near-perfect repayment rates (`99.7%`) while unemployed borrowers defaulted at alarming rates (`92.2%`). This `91.9` percentage point difference represents one of the most significant risk differentials observed in credit modeling literature and suggests employment dynamics are substantially underweighted in conventional risk frameworks. Our correlation analysis established two distinct risk clusters. The financial capacity cluster, centered around DTI (`-0.34` correlation with repayment), captures borrowers' ability to service debt obligations. The creditworthiness cluster, represented by credit score (`0.23` correlation), reflects historical repayment behavior. These clusters operate largely independently (Cramer's V < `0.05`), creating modeling opportunities through carefully constructed interaction terms. The grade_subgrade analysis revealed a systematic risk progression across credit tiers (A=`4.8%`, B=`6.8%`, C=`15.3`%, D=`28.5%` default rates), validating the theoretical foundation of credit grading systems while highlighting non-linear risk jumps at critical grade boundaries, particularly the C-to-D transition. For the grade_subgrade feature, we resolved the encoding dilemma through a hybrid approach that preserved ordinal information while accommodating non-linearities. We mapped each subgrade to an ordinal scale based on default rate progression, then supplemented this with polynomial transformations to capture disproportionate risk jumps at grade boundaries. This strategy maintained the interpretability of the grading system while capturing its complex risk properties.



> 👀 It might be useful to isolate the age based on `education_level` and `employment_status`, but this seems to be a chaotic solution, since the data is synthetic.


Given the competition's ROC AUC evaluation metric and the dataset's characteristics, we implemented a sophisticated modeling framework centered on CatBoost with GPU-accelerated Optuna optimization. This approach leveraged several key advantages of gradient boosting for credit risk assessment: native handling of mixed data types, robustness to outliers, and automatic feature interaction detection.

<div style="color: white;
    display: flex;
    justify-content: center;
    align-items: center;
    border-radius: 5px;
    background-color: #003247;
    font-size: 220%;
    font-family: Nexa;
    letter-spacing: 0.5px;
    min-height: 100px;
    padding: 20px; 
    width: 100%;
    box-sizing: border-box;">
    <h1 id = "refs"
        style=" color: white;
        margin: 0;
        text-align: center;
        width: 100%;">
        <b>References 📜</b>
    </h1>
</div>

1. [ROC AUC curve explained |Do it with your own hands](https://www.kaggle.com/code/meowmeowmeowmeowmeow/roc-auc-curve-explained-do-it-with-your-own-hands/notebook)
2. [CatBoost-LightGBM-XGBoost Explained by SHAP](https://www.kaggle.com/code/kaanboke/catboost-lightgbm-xgboost-explained-by-shap)
3. [Simple XGBoost | Only Competition Data – S5E11](https://www.kaggle.com/code/yousefelshahat2/simple-xgboost-only-competition-data-s5e11/notebook)
4. [Cramer's V correlation matrix](https://www.kaggle.com/code/chrisbss1/cramer-s-v-correlation-matrix)
5. [Loan Approval | EDA + Catboost + Optuna](https://www.kaggle.com/code/igorvolianiuk/loan-approval-eda-catboost-optuna)
6. [S4:E10|Loan Status Prediction-CatBoost 97% 🎯📊](https://www.kaggle.com/code/rv1922/s4-e10-loan-status-prediction-catboost-97)
7. [Loan Prediction Dataset ML Project 📈](https://www.kaggle.com/code/yonatanrabinovich/loan-prediction-dataset-ml-project#The-Process-of-Modeling-the-Data:)